<a href="https://colab.research.google.com/github/Tamim-saad/DL_practice/blob/master/Hidden_Layers_And_Hidden_Neurons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Keras Tuner- Decide Number of Hidden Layers And Neuron In Neural Network

In [48]:
# Mount Google Drive (for Google Colab)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
# Install Keras Tuner FIRST
!pip install -q -U keras-tuner

# Then import everything
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [50]:
df=pd.read_csv('/content/drive/MyDrive/Personal-Important/DL_playlist_code/24/Real_Combine.csv')

In [51]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [52]:
X=df.iloc[:,:-1] ## independent features
y=df.iloc[:,-1] ## dependent features

In [53]:
# Remove infinite values
X.replace([np.inf, -np.inf], np.nan, inplace=True)
y.replace([np.inf, -np.inf], np.nan, inplace=True)

#1// wrong bcz not following synchronization
# X = X.dropna()
# y= y.dropna()

#2
df_clean = pd.concat([X, y], axis=1).dropna()
X = df_clean.iloc[:, :-1]
y = df_clean.iloc[:, -1]

/tmp/ipython-input-3499704051.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.replace([np.inf, -np.inf], np.nan, inplace=True)


In [54]:
# Split FIRST (important!)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [55]:
# Then scale
sc_X = StandardScaler()
sc_y = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
y_train = sc_y.fit_transform(y_train.values.reshape(-1, 1))
y_test = sc_y.transform(y_test.values.reshape(-1, 1))

In [56]:
# Verify clean data
print(f"NaN in X_train: {np.isnan(X_train).any()}")
print(f"NaN in y_train: {np.isnan(y_train).any()}")

NaN in X_train: False
NaN in y_train: False


#### Hyperparameters
1. How many number of hidden layers we should have?
2. How many number of neurons we should have in hidden layers?
3. Learning Rate

In [57]:
def build_model(hp):
    model = keras.Sequential()

      # ADD THIS: Specify input shape explicitly
    model.add(layers.Input(shape=(X_train.shape[1],)))

    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error', keras.metrics.R2Score()])
    return model

In [58]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')

Reloading Tuner from project/Air Quality Index/tuner0.json


In [59]:
tuner.search_space_summary()

Search space summary
Default search space size: 21
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_6

In [60]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

First, let's explicitly retrieve the best model (which we already did, but this ensures we have it for further operations) and then access its training history. The `tuner.search()` method stores the history of the trials. We can get the details of the best trial and then access its run details.

The `val_mean_absolute_error` is the objective metric, and the tuner uses this to determine which model is 'best'. The `get_best_hyperparameters` and `get_best_models` methods then return the trial that minimized this value. If the plot is generated, you will see how this metric evolved during the training of the best trial.

In [61]:
# 1. Print search results summary
# print("=" * 60)
# print("SEARCH RESULTS SUMMARY")
# print("=" * 60)
# tuner.results_summary(num_trials=3)  # Shows top 5 trials

# 2. Get best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("\n" + "=" * 60)
print("BEST HYPERPARAMETERS")
print("=" * 60)
print(f"Number of layers: {best_hps.get('num_layers')}")
print(f"Learning rate: {best_hps.get('learning_rate')}")
for i in range(best_hps.get('num_layers')):
    print(f"Layer {i+1} units: {best_hps.get(f'units_{i}')}")

# 3. Get the best trained model
best_model = tuner.get_best_models(num_models=1)[0]
print("\n" + "=" * 60)
print("BEST MODEL ARCHITECTURE")
print("=" * 60)
best_model.summary()



BEST HYPERPARAMETERS
Number of layers: 8
Learning rate: 0.001
Layer 1 units: 224
Layer 2 units: 96
Layer 3 units: 96
Layer 4 units: 512
Layer 5 units: 512
Layer 6 units: 480
Layer 7 units: 384
Layer 8 units: 128

BEST MODEL ARCHITECTURE


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 224)            │         2,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 96)             │        21,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 96)             │         9,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 512)            │        49,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 480)            │       246,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 384)            │       184,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 825,601 (3.15 MB)

 Trainable params: 825,601 (3.15 MB)

 Non-trainable params: 0 (0.00 B)

In [62]:
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model on the test data
loss, mae, r2_score_value = best_model.evaluate(X_test, y_test, verbose=1)

print(f"\nEvaluation of the Best Model on Test Data:")
print(f"  Loss (Mean Absolute Error): {mae:.4f}")
print(f"  R2 Score: {r2_score_value:.4f}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.4115 - mean_absolute_error: 0.4115 - r2_score: 0.6592

Evaluation of the Best Model on Test Data:
  Loss (Mean Absolute Error): 0.4129
  R2 Score: 0.6468
